# 13-9. 프로젝트 - Movielens 영화 추천 실습


Movielens 데이터  
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

In [1]:
import os

import pandas as pd
import numpy as np
import scipy
import implicit
from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares


print(implicit.__version__)

0.4.8


## 1) 데이터 준비와 전처리


Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# 사용자 정보을 보기 위해 메타 데이터를 읽어옵니다.
user_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/users.dat'
cols = ['user_id', 'gender', 'temp1','temp2','temp3'] 
users = pd.read_csv(user_file_path, sep='::'
                    , names=cols
                    , engine='python', encoding='ISO-8859-1')
print(len(users))
users.head()


6040


,user_id,gender,temp1,temp2,temp3
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


users에는 이번 학습에 영향을 주는 데이터가 없네요

In [8]:
# ratings, movie, users 합치기 -> total_df

total_df = pd.merge(ratings, movies, on='movie_id', how='left')

In [9]:
total_df.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


title 을 깔끔하게 보기 위해 상영 연도를 지우고, 이번 실습 목적에 맞게 user_id, title, counts만 남기고 모두 제거하겠습니다.

In [10]:
# 필요없는 칼럼 삭제
ratings = total_df.drop(['movie_id', 'timestamp', 'genre'], axis=1)

# title의 연도 지우기
ratings['title'] = ratings['title'].apply(lambda x: x[:-7])

In [11]:
ratings.head()

,user_id,counts,title
0,1,5,One Flew Over the Cuckoo's Nest
1,1,3,James and the Giant Peach
2,1,3,My Fair Lady
3,1,4,Erin Brockovich
4,1,5,"Bug's Life, A"


## 2) 분석해 봅시다.


- ratings에 있는 유니크한 영화 개수


In [12]:
# 유니크 영화개수
ratings['title'].nunique()

3586

- ratings에 있는 유니크한 사용자 수


In [13]:
# 유니크한 사용자 수
ratings['user_id'].nunique()

6039

- 가장 인기 있는 영화 30개(인기순)

In [14]:
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty                                   3211
Star Wars: Episode IV - A New Hope                2910
Star Wars: Episode V - The Empire Strikes Back    2885
Star Wars: Episode VI - Return of the Jedi        2716
Saving Private Ryan                               2561
Terminator 2: Judgment Day                        2509
Silence of the Lambs, The                         2498
Raiders of the Lost Ark                           2473
Back to the Future                                2460
Matrix, The                                       2434
Jurassic Park                                     2413
Sixth Sense, The                                  2385
Fargo                                             2371
Braveheart                                        2314
Men in Black                                      2297
Schindler's List                                  2257
Princess Bride, The                               2252
Shakespeare in Love                               2213
L.A.

## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.


In [15]:
# 선호하는 영화 5가지 골라서 추가하기
my_favorite = ['Sixth Sense, The' , 'Blade Runner' ,'E.T. the Extra-Terrestrial' ,'Matrix, The' ,'Gladiator']

# users 에 내정보 추가 : user_id 는 'jtkim'으로 추가하겠습니다.
# 영화의 평점은 5점씩 줬다고 가정.
my_ratings = pd.DataFrame({'user_id': ['jtkim']*5, 'title': my_favorite, 'counts': [5]*5})

if not ratings.isin({'user_id':['jtkim']})['user_id'].any():    # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_ratings)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,counts,title
836473,6040,3,Platoon
836474,6040,5,"Crying Game, The"
836475,6040,5,Welcome to the Dollhouse
836476,6040,4,Sophie's Choice
836477,6040,4,E.T. the Extra-Terrestrial
0,jtkim,5,"Sixth Sense, The"
1,jtkim,5,Blade Runner
2,jtkim,5,E.T. the Extra-Terrestrial
3,jtkim,5,"Matrix, The"
4,jtkim,5,Gladiator


## 4) CSR matrix를 직접 만들어 봅시다.


- CSR 
  - ratings.user_id
  - ratings.movie_id
  - ratings.counts 



In [16]:
# 고유한 유저, 영화 찾는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_id_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_id_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [17]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_id_to_idx['jtkim'])    
print(movie_id_to_idx['Blade Runner'])

6039
676


In [18]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_id_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_id_to_idx 를 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = ratings['title'].map(movie_id_to_idx.get).dropna()
if len(temp_title_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,counts,title
0,0,5,0
1,0,3,1
2,0,3,2
3,0,4,3
4,0,5,4
...,...,...,...
0,6039,5,38
1,6039,5,676
2,6039,5,26
3,6039,5,124


In [19]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3586 sparse matrix of type '<class 'numpy.int64'>'
	with 834213 stored elements in Compressed Sparse Row format>

## 5) AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.


In [20]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 init 파라미터

- factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
- regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
- use_gpu : GPU를 사용할 것인지
- iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

In [21]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100
                                    , regularization=0.01
                                    , use_gpu=False
                                    , iterations=15  #epoch
                                    , dtype=np.float32
                                   )

In [22]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3586x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 834213 stored elements in Compressed Sparse Column format>

In [23]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.


In [24]:
# blade_runner와의 선호도 조사
myid, blade_runner = user_id_to_idx['jtkim'], movie_id_to_idx['Blade Runner']    
myid_vector, blade_runner_vector = als_model.user_factors[myid], als_model.item_factors[blade_runner]
np.dot(myid_vector, blade_runner_vector)

0.44589403

영화의 선호도는 0.45정도로 작게 나왔습니다.

In [42]:
# 벡터 살펴보기
print(len(myid_vector))
myid_vector

100


array([-3.24977130e-01, -1.19394697e-01, -7.46301174e-01, -2.45933726e-01,
        1.64863512e-01, -3.61543804e-01, -9.79064941e-01, -5.33214211e-01,
       -4.19719994e-01,  4.18325663e-01,  7.96327233e-01,  5.09014845e-01,
       -1.22957513e-01, -7.75169909e-01,  6.70604110e-01, -9.42979157e-01,
       -3.20040971e-01,  3.12115401e-01,  6.11353517e-01, -2.58416593e-01,
        2.95872480e-01,  1.97605386e-01,  5.90662062e-01, -2.13441238e-01,
       -2.38253608e-01,  1.00875735e-01,  1.71050102e-01, -3.75525266e-01,
       -7.40274191e-01, -5.95644474e-01, -3.74566734e-01,  1.50425649e+00,
        8.72083679e-02,  2.38723651e-01,  2.11015433e-01,  5.23512602e-01,
        5.10532856e-01,  4.27586645e-01, -9.32792723e-02, -4.84426677e-01,
       -9.60310027e-02,  6.56233251e-01, -9.61580515e-01,  6.47515416e-01,
       -2.28026435e-01,  4.10660774e-01, -1.73608765e-01, -4.42298710e-01,
       -7.07598776e-02, -1.88090280e-02, -3.83341402e-01,  1.59869060e-01,
        2.44314782e-02, -

In [44]:
print(blade_runner_vector.shape)
blade_runner_vector

(100,)


array([ 1.53260734e-02, -6.34238264e-03,  7.61292549e-03, -5.23720346e-02,
        3.62354405e-02, -1.72227365e-03, -1.98763590e-02, -8.38518888e-03,
        1.12484032e-02,  1.34450058e-02,  1.79978088e-02,  2.01587360e-02,
        6.37268880e-04, -3.93640399e-02,  2.17739772e-02,  9.99022275e-03,
       -1.13224061e-02,  2.06819307e-02,  3.01403776e-02, -1.74234863e-02,
        2.10723523e-02,  1.38937659e-03, -3.09627457e-03, -8.51127505e-03,
        1.73236127e-03,  1.06582865e-02,  8.15299992e-03, -1.29253604e-02,
       -8.65885708e-03,  9.08248650e-04, -2.67288312e-02,  3.66512164e-02,
        1.28841102e-02, -3.22185690e-03,  3.82014387e-03,  2.13699974e-03,
        8.56345613e-03,  3.43214758e-02, -5.46856783e-03,  9.91615350e-04,
        3.38278798e-04,  6.75647566e-03, -4.06468846e-03,  2.46602260e-02,
        1.23941787e-02,  8.90651718e-03,  1.21946801e-02, -2.44310871e-02,
       -8.89317319e-03,  2.17966409e-03,  5.00119757e-03, -3.03213077e-04,
        1.95639636e-02, -

In [25]:
# 다른 영화 (Men in Black) 과의 선호도 조사 

other_movie = movie_id_to_idx['Men in Black']                   # movie_id : 541 = 'Blade Runner (1982)'
othermovie_vector = als_model.item_factors[other_movie]
np.dot(myid_vector, othermovie_vector)

0.08460458

선택하지 않은 영화는 선호도가 확실히 낮게 나오긴 하네요

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.


In [26]:
favorite_movie = 'Matrix, The'           # 메트릭스와 비슷한 영화를 추천받기
movie_id = movie_id_to_idx[favorite_movie]
similar_movies = als_model.similar_items(movie_id, N=15)
similar_movies                    

[(124, 0.99999994),
 (92, 0.7947834),
 (62, 0.6407718),
 (141, 0.6010488),
 (200, 0.58655167),
 (145, 0.5192738),
 (107, 0.51505303),
 (375, 0.49242175),
 (175, 0.46254376),
 (75, 0.4462058),
 (317, 0.4348753),
 (44, 0.4025045),
 (430, 0.3849527),
 (236, 0.38163343),
 (3416, 0.36358827)]

In [28]:
# movie_id_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_id_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movies]

['Matrix, The',
 'Terminator 2: Judgment Day',
 'Total Recall',
 'Fugitive, The',
 'Terminator, The',
 'Fifth Element, The',
 'Jurassic Park',
 'Face/Off',
 'Men in Black',
 'Hunt for Red October, The',
 'Twelve Monkeys',
 'Star Wars: Episode IV - A New Hope',
 'Gattaca',
 'Speed',
 'Second Jungle Book: Mowgli & Baloo, The']

제가 다 좋아하는 영화들입니다. 추천을 잘 해주는 것 같습니다.

In [29]:
# 함수로 만들기 
def get_similar_movie(favorite_movie: str):
    movie_id = movie_id_to_idx[favorite_movie]
    similar_movies = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movies]
    return similar_movie

In [30]:
# 함수 확인
get_similar_movie('Face/Off')

['Face/Off',
 'Fifth Element, The',
 'Negotiator, The',
 'Enemy of the State',
 'Rock, The',
 'Total Recall',
 'Long Kiss Goodnight, The',
 'Matrix, The',
 'Independence Day (ID4)',
 'Con Air']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.


In [31]:
csr_data

<6040x3586 sparse matrix of type '<class 'numpy.int64'>'
	with 834213 stored elements in Compressed Sparse Row format>

In [34]:
user_idx = user_id_to_idx['jtkim']

# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user_idx
                                         , csr_data
                                         , N=20, filter_already_liked_items=True
                                         )
movie_recommended

[(92, 0.43914154),
 (200, 0.40091097),
 (346, 0.34043658),
 (22, 0.33604246),
 (224, 0.3159176),
 (372, 0.31247795),
 (647, 0.2967812),
 (599, 0.28381777),
 (317, 0.28092152),
 (44, 0.28033403),
 (481, 0.27538204),
 (60, 0.2685219),
 (117, 0.26777464),
 (91, 0.2660874),
 (121, 0.26513904),
 (273, 0.26198155),
 (193, 0.25806588),
 (233, 0.24928221),
 (99, 0.2441332),
 (478, 0.24320519)]

In [35]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Terminator 2: Judgment Day',
 'Terminator, The',
 'Patriot, The',
 'Back to the Future',
 'L.A. Confidential',
 'X-Men',
 'Aliens',
 '2001: A Space Odyssey',
 'Twelve Monkeys',
 'Star Wars: Episode IV - A New Hope',
 'Mission: Impossible 2',
 'Star Wars: Episode I - The Phantom Menace',
 'Star Wars: Episode V - The Empire Strikes Back',
 'Close Encounters of the Third Kind',
 'Silence of the Lambs, The',
 'Fight Club',
 'Alien',
 'Usual Suspects, The',
 'American Beauty',
 'Contact']

X-men 추천해 주고 있네요. explain 메서드를 사용해 제가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인해 보겠습니다.

In [39]:
X_Men = movie_id_to_idx['X-Men']
explain = als_model.explain(user_idx, csr_data, itemid=X_Men)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Gladiator', 0.18522010910303788),
 ('Matrix, The', 0.11449544651806609),
 ('Blade Runner', 0.0365091550881506),
 ('Sixth Sense, The', -0.01276246777741988),
 ('E.T. the Extra-Terrestrial', -0.014417530848555713)]

Gladiator와 Matrix가 큰 기여를 했네요. 이번 주말에는 X-men을 봐야겠습니다. 

## 회고

- user X item 행렬을 CSR 행렬로 만들어 데이터량을 줄이고 예측을 빠르게 할 수 있다는 아이디어에 감탄했습니다.
- Matrix Factorization도 간단한 라이브러리를 이용해 구성할 수 있다니 신기했습니다.
- ratings의 timestpemp나 movies의 genre 또는 users에 있는 성별, 나이? 등의 다른 데이터도 추천에 고려할 수 있는 방법이 있을지 고민해 보고 싶었습니다.
- 단순히 2차원(행렬) 데이터로 만드는 것이 아닌 다차원의 데이터도 가능하지 않을까 생각해 봤습니다. 예를들어 영화, 책, 음악, 게임 등을 추천 할 때 소비자가 각각 어떤 성격의 컨텐츠를 소비하는 지에 대한 데이터가 있다면 다차원의 MF를 구성할 수 있지 않을까하는 생각입니다.
  
  
<br>
감사합니다.